In [4]:
"""
END-TO-END PROCUREMENT-TO-REVENUE ANALYTICS & INSIGHTS STUDY
Part 2: Advanced Analytics & Machine Learning

This notebook performs predictive analytics and advanced statistical analysis
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, classification_report, confusion_matrix
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("="*80)
print("ADVANCED ANALYTICS & PREDICTIVE MODELING")
print("="*80)

# ============================================================================
# DATABASE CONNECTION & DATA LOADING
# ============================================================================

DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',  # CHANGE THIS
    'database': 'hyfun_analytics'
}

connection_string = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

print("\n[1/8] Loading data...")

# Load necessary tables
b2b_customers = pd.read_sql("SELECT * FROM b2b_customers", engine)
b2b_orders = pd.read_sql("SELECT * FROM b2b_orders", engine)
production = pd.read_sql("SELECT * FROM production_batches", engine)
products = pd.read_sql("SELECT * FROM product_master", engine)
quality = pd.read_sql("SELECT * FROM quality_control", engine)
b2c_sales = pd.read_sql("SELECT * FROM b2c_sales", engine)

print("✓ Data loaded successfully")

ADVANCED ANALYTICS & PREDICTIVE MODELING

[1/8] Loading data...
✓ Data loaded successfully


In [6]:
# ============================================================================
# ANALYSIS 1: CUSTOMER LIFETIME VALUE (CLV) PREDICTION
# ============================================================================

print("\n[2/8] Building Customer Lifetime Value Model...")
print("\n" + "="*80)
print("CUSTOMER LIFETIME VALUE PREDICTION")
print("="*80)

# Prepare customer features
b2b_orders['order_date'] = pd.to_datetime(b2b_orders['order_date'])

customer_features = b2b_orders.groupby('customer_id').agg({
    'order_id': 'count',
    'total_value_inr': ['sum', 'mean', 'std'],
    'quantity_kg': ['sum', 'mean'],
    'order_date': ['min', 'max']
}).reset_index()

customer_features.columns = ['customer_id', 'total_orders', 'total_revenue', 
                             'avg_order_value', 'std_order_value', 'total_quantity',
                             'avg_quantity', 'first_order', 'last_order']

# Calculate additional features
customer_features['tenure_days'] = (customer_features['last_order'] - customer_features['first_order']).dt.days
customer_features['days_since_last_order'] = (pd.Timestamp.now() - customer_features['last_order']).dt.days
customer_features['order_frequency'] = customer_features['total_orders'] / (customer_features['tenure_days'] + 1) * 30
customer_features['std_order_value'] = customer_features['std_order_value'].fillna(0)

# Merge with customer data
customer_features = customer_features.merge(
    b2b_customers[['customer_id', 'customer_type', 'country', 'credit_period_days']], 
    on='customer_id'
)

# Encode categorical variables
le_type = LabelEncoder()
le_country = LabelEncoder()
customer_features['customer_type_encoded'] = le_type.fit_transform(customer_features['customer_type'])
customer_features['country_encoded'] = le_country.fit_transform(customer_features['country'])

# Features for modeling
feature_cols = ['total_orders', 'avg_order_value', 'avg_quantity', 'tenure_days',
                'days_since_last_order', 'order_frequency', 'customer_type_encoded',
                'country_encoded', 'credit_period_days', 'std_order_value']

X = customer_features[feature_cols].fillna(0)
y = customer_features['total_revenue']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
rf_clv = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_clv.fit(X_train, y_train)

# Predictions
y_pred = rf_clv.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n📊 CLV Model Performance:")
print(f"Mean Absolute Error: ₹{mae:,.0f}")
print(f"R² Score: {r2:.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_clv.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n📊 Top 5 CLV Predictors:")
print(feature_importance.head())

# Predict CLV for all customers
customer_features['predicted_clv'] = rf_clv.predict(X)
customer_features['clv_segment'] = pd.qcut(customer_features['predicted_clv'], 
                                            q=4, labels=['Low', 'Medium', 'High', 'Very High'])

print(f"\n📊 CLV Segmentation:")
print(customer_features['clv_segment'].value_counts().sort_index())

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Actual vs Predicted
axes[0].scatter(y_test / 100000, y_pred / 100000, alpha=0.5)
axes[0].plot([y_test.min() / 100000, y_test.max() / 100000], 
             [y_test.min() / 100000, y_test.max() / 100000], 'r--', lw=2)
axes[0].set_xlabel('Actual CLV (Lakhs)')
axes[0].set_ylabel('Predicted CLV (Lakhs)')
axes[0].set_title('CLV Prediction: Actual vs Predicted', fontweight='bold')
axes[0].text(0.05, 0.95, f'R² = {r2:.3f}', transform=axes[0].transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Feature importance
feature_importance.head(8).sort_values('importance').plot(kind='barh', x='feature', y='importance', 
                                                            ax=axes[1], legend=False, color='teal')
axes[1].set_xlabel('Importance')
axes[1].set_title('Feature Importance for CLV', fontweight='bold')

# CLV distribution by segment
customer_features.boxplot(column='predicted_clv', by='clv_segment', ax=axes[2])
axes[2].set_xlabel('CLV Segment')
axes[2].set_ylabel('Predicted CLV (₹)')
axes[2].set_title('CLV Distribution by Segment', fontweight='bold')
axes[2].get_figure().suptitle('')

plt.tight_layout()
plt.savefig('07_clv_prediction.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: 07_clv_prediction.png")
plt.close()


[2/8] Building Customer Lifetime Value Model...

CUSTOMER LIFETIME VALUE PREDICTION

📊 CLV Model Performance:
Mean Absolute Error: ₹334,036
R² Score: 0.980

📊 Top 5 CLV Predictors:
           feature  importance
0     total_orders    0.774781
5  order_frequency    0.129066
1  avg_order_value    0.069988
2     avg_quantity    0.008908
9  std_order_value    0.005128

📊 CLV Segmentation:
clv_segment
Low          50
Medium       50
High         50
Very High    50
Name: count, dtype: int64

✓ Saved: 07_clv_prediction.png


In [7]:
# ============================================================================
# ANALYSIS 2: CHURN PREDICTION
# ============================================================================

print("\n[3/8] Building Churn Prediction Model...")
print("\n" + "="*80)
print("CUSTOMER CHURN PREDICTION")
print("="*80)

# Define churn based on data distribution
# Use median of days_since_last_order as threshold
churn_threshold = customer_features['days_since_last_order'].median()
print(f"\n📊 Churn threshold (median): {churn_threshold:.0f} days")

customer_features['churned'] = (customer_features['days_since_last_order'] > churn_threshold).astype(int)

print(f"\n📊 Churn Rate: {customer_features['churned'].mean() * 100:.2f}%")
print(f"Churned Customers: {customer_features['churned'].sum()}")
print(f"Active Customers: {(1 - customer_features['churned']).sum()}")

# Features for churn prediction
churn_features = ['total_orders', 'avg_order_value', 'tenure_days', 
                  'order_frequency', 'customer_type_encoded', 'country_encoded',
                  'credit_period_days', 'std_order_value']

X_churn = customer_features[churn_features].fillna(0)
y_churn = customer_features['churned']

# Split data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_churn, y_churn, 
                                                              test_size=0.2, random_state=42, stratify=y_churn)

# Train model
rf_churn = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=8)
rf_churn.fit(X_train_c, y_train_c)

# Predictions
y_pred_churn = rf_churn.predict(X_test_c)

# Check if model can predict probabilities for both classes
try:
    y_pred_proba = rf_churn.predict_proba(X_test_c)[:, 1]
except IndexError:
    print("\n⚠️ Warning: Only one class present in predictions. Using predicted labels instead.")
    y_pred_proba = y_pred_churn.astype(float)

# Evaluate
print(f"\n📊 Churn Prediction Performance:")
print(classification_report(y_test_c, y_pred_churn, target_names=['Active', 'Churned']))

# Predict churn probability for all customers
try:
    customer_features['churn_probability'] = rf_churn.predict_proba(X_churn)[:, 1]
except IndexError:
    # If only one class, use predictions as probability
    customer_features['churn_probability'] = rf_churn.predict(X_churn).astype(float)
    
customer_features['churn_risk'] = pd.cut(customer_features['churn_probability'],
                                          bins=[0, 0.3, 0.6, 1.0],
                                          labels=['Low', 'Medium', 'High'])

print(f"\n📊 Churn Risk Distribution:")
print(customer_features['churn_risk'].value_counts())

# High-value at-risk customers
at_risk_valuable = customer_features[
    (customer_features['churn_risk'] == 'High') & 
    (customer_features['total_revenue'] > customer_features['total_revenue'].quantile(0.75))
].sort_values('total_revenue', ascending=False)

print(f"\n⚠️ High-Value At-Risk Customers: {len(at_risk_valuable)}")
print(f"Revenue at Risk: ₹{at_risk_valuable['total_revenue'].sum() / 10000000:.2f} Cr")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Confusion Matrix
cm = confusion_matrix(y_test_c, y_pred_churn)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0],
            xticklabels=['Active', 'Churned'], yticklabels=['Active', 'Churned'])
axes[0, 0].set_title('Confusion Matrix', fontweight='bold')
axes[0, 0].set_ylabel('Actual')
axes[0, 0].set_xlabel('Predicted')

# Feature importance
churn_importance = pd.DataFrame({
    'feature': churn_features,
    'importance': rf_churn.feature_importances_
}).sort_values('importance', ascending=False)

churn_importance.head(8).sort_values('importance').plot(kind='barh', x='feature', y='importance',
                                                          ax=axes[0, 1], legend=False, color='coral')
axes[0, 1].set_xlabel('Importance')
axes[0, 1].set_title('Churn Prediction Features', fontweight='bold')

# Churn risk by customer type
risk_by_type = customer_features.groupby(['customer_type', 'churn_risk']).size().unstack(fill_value=0)
risk_by_type.plot(kind='bar', stacked=True, ax=axes[1, 0])
axes[1, 0].set_title('Churn Risk by Customer Type', fontweight='bold')
axes[1, 0].set_xlabel('Customer Type')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].legend(title='Risk')

# Revenue at risk
revenue_risk = customer_features.groupby('churn_risk')['total_revenue'].sum() / 10000000
revenue_risk.plot(kind='bar', ax=axes[1, 1], color=['green', 'orange', 'red'])
axes[1, 1].set_title('Revenue by Churn Risk', fontweight='bold')
axes[1, 1].set_ylabel('Revenue (Crores)')
axes[1, 1].set_xlabel('Risk Level')
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('08_churn_prediction.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: 08_churn_prediction.png")
plt.close()


[3/8] Building Churn Prediction Model...

CUSTOMER CHURN PREDICTION

📊 Churn threshold (median): 395 days

📊 Churn Rate: 48.00%
Churned Customers: 96
Active Customers: 104

📊 Churn Prediction Performance:
              precision    recall  f1-score   support

      Active       0.75      0.71      0.73        21
     Churned       0.70      0.74      0.72        19

    accuracy                           0.72        40
   macro avg       0.72      0.73      0.72        40
weighted avg       0.73      0.72      0.73        40


📊 Churn Risk Distribution:
churn_risk
High      89
Low       82
Medium    29
Name: count, dtype: int64

⚠️ High-Value At-Risk Customers: 14
Revenue at Risk: ₹15.30 Cr

✓ Saved: 08_churn_prediction.png


In [13]:
# ============================================================================
# FIXED: DEMAND FORECASTING (Using ALL data, no recent filter)
# ============================================================================

print("\n[4/8] Building Demand Forecasting Model...")

# Load ALL orders
b2b_orders['order_date'] = pd.to_datetime(b2b_orders['order_date'])

# Aggregate to WEEKLY level (more stable than daily)
weekly_demand = b2b_orders.set_index('order_date').resample('W')['quantity_kg'].sum().reset_index()
weekly_demand.columns = ['date', 'quantity_kg']

# Only proceed if we have enough data
if len(weekly_demand) < 20:
    print(f"\n⚠️ Insufficient data for forecasting: Only {len(weekly_demand)} weeks available")
    print("Need at least 20 weeks. Skipping demand forecasting...")
else:
    # Create features
    weekly_demand['week_of_year'] = weekly_demand['date'].dt.isocalendar().week
    weekly_demand['month'] = weekly_demand['date'].dt.month
    weekly_demand['quarter'] = weekly_demand['date'].dt.quarter
    
    # Rolling statistics
    weekly_demand['rolling_mean_4'] = weekly_demand['quantity_kg'].rolling(window=4, min_periods=1).mean()
    weekly_demand['rolling_std_4'] = weekly_demand['quantity_kg'].rolling(window=4, min_periods=1).std()
    
    # Lag features
    weekly_demand['lag_1'] = weekly_demand['quantity_kg'].shift(1)
    weekly_demand['lag_4'] = weekly_demand['quantity_kg'].shift(4)
    
    # Drop NaN
    weekly_clean = weekly_demand.dropna()
    
    if len(weekly_clean) < 10:
        print(f"\n⚠️ After cleaning: Only {len(weekly_clean)} weeks available. Skipping...")
    else:
        # Features and target
        forecast_features = ['week_of_year', 'month', 'quarter', 'rolling_mean_4', 'rolling_std_4', 'lag_1', 'lag_4']
        X_demand = weekly_clean[forecast_features]
        y_demand = weekly_clean['quantity_kg']
        
        # Split (80-20)
        split_idx = int(len(X_demand) * 0.8)
        X_train_d = X_demand[:split_idx]
        X_test_d = X_demand[split_idx:]
        y_train_d = y_demand[:split_idx]
        y_test_d = y_demand[split_idx:]
        
        # Train
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.metrics import mean_absolute_error, r2_score
        
        rf_demand = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=8)
        rf_demand.fit(X_train_d, y_train_d)
        
        # Predict
        y_pred_demand = rf_demand.predict(X_test_d)
        
        # Evaluate
        mae_demand = mean_absolute_error(y_test_d, y_pred_demand)
        r2_demand = r2_score(y_test_d, y_pred_demand)
        
        print(f"\n📊 Weekly Demand Forecast Performance:")
        print(f"Mean Absolute Error: {mae_demand:,.0f} kg/week")
        print(f"R² Score: {r2_demand:.3f}")
        print(f"Average Weekly Demand: {y_demand.mean():,.0f} kg")
        
        # Visualization
        import matplotlib.pyplot as plt
        
        fig, ax = plt.subplots(1, 1, figsize=(14, 6))
        test_dates = weekly_clean['date'].iloc[split_idx:]
        ax.plot(test_dates.values, y_test_d.values, label='Actual', linewidth=2, marker='o')
        ax.plot(test_dates.values, y_pred_demand, label='Predicted', linewidth=2, marker='s')
        ax.set_title('Weekly Demand Forecast', fontsize=14, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel('Weekly Demand (kg)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('09_demand_forecasting.png', dpi=300, bbox_inches='tight')
        print("\n✓ Saved: 09_demand_forecasting.png")
        plt.close()


[4/8] Building Demand Forecasting Model...

📊 Weekly Demand Forecast Performance:
Mean Absolute Error: 10,571 kg/week
R² Score: -0.019
Average Weekly Demand: 61,569 kg

✓ Saved: 09_demand_forecasting.png


In [9]:
# ============================================================================
# ANALYSIS 4: PRODUCT RECOMMENDATION (MARKET BASKET)
# ============================================================================

print("\n[5/8] Market Basket Analysis...")
print("\n" + "="*80)
print("PRODUCT AFFINITY ANALYSIS")
print("="*80)

# Find products bought together
order_products = b2b_orders.groupby('order_id')['product_sku'].apply(list).reset_index()

# Count co-occurrences
from itertools import combinations

product_pairs = []
for products_list in order_products['product_sku']:
    if len(products_list) > 1:
        for pair in combinations(sorted(products_list), 2):
            product_pairs.append(pair)

pair_counts = pd.DataFrame(product_pairs, columns=['product_a', 'product_b'])
pair_counts = pair_counts.groupby(['product_a', 'product_b']).size().reset_index(name='count')
pair_counts = pair_counts.sort_values('count', ascending=False)

# Merge with product names
pair_counts = pair_counts.merge(products[['product_sku', 'product_name']], 
                                 left_on='product_a', right_on='product_sku', how='left')
pair_counts = pair_counts.rename(columns={'product_name': 'product_a_name'}).drop('product_sku', axis=1)

pair_counts = pair_counts.merge(products[['product_sku', 'product_name']], 
                                 left_on='product_b', right_on='product_sku', how='left')
pair_counts = pair_counts.rename(columns={'product_name': 'product_b_name'}).drop('product_sku', axis=1)

print(f"\n📊 Top 10 Product Pairs Bought Together:")
top_pairs = pair_counts.head(10)[['product_a_name', 'product_b_name', 'count']]
for idx, row in top_pairs.iterrows():
    print(f"{row['product_a_name']} + {row['product_b_name']}: {row['count']} orders")

# Visualization
plt.figure(figsize=(14, 8))
top_pairs['pair'] = top_pairs['product_a_name'].str[:15] + ' +\n' + top_pairs['product_b_name'].str[:15]
plt.barh(range(len(top_pairs)), top_pairs['count'], color='steelblue')
plt.yticks(range(len(top_pairs)), top_pairs['pair'])
plt.xlabel('Co-occurrence Count')
plt.title('Top 10 Product Combinations', fontsize=16, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('10_product_affinity.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: 10_product_affinity.png")
plt.close()


[5/8] Market Basket Analysis...

PRODUCT AFFINITY ANALYSIS

📊 Top 10 Product Pairs Bought Together:

✓ Saved: 10_product_affinity.png


In [10]:
# ============================================================================
# ANALYSIS 5: QUALITY PREDICTION
# ============================================================================

print("\n[6/8] Quality Score Prediction...")
print("\n" + "="*80)
print("BRC COMPLIANCE PREDICTION")
print("="*80)

# Merge production with quality
quality['inspection_date'] = pd.to_datetime(quality['inspection_date'])
production['production_date'] = pd.to_datetime(production['production_date'])

quality_prod = quality.merge(production[['batch_id', 'product_sku', 'plant_location', 
                                          'shift', 'raw_material_used_mt', 'finished_goods_mt',
                                          'processing_time_hours']], on='batch_id')

# Calculate conversion rate
quality_prod['conversion_rate'] = (quality_prod['finished_goods_mt'] / 
                                    quality_prod['raw_material_used_mt'] * 100)

# Encode categoricals
le_plant = LabelEncoder()
le_shift = LabelEncoder()
le_product = LabelEncoder()

quality_prod['plant_encoded'] = le_plant.fit_transform(quality_prod['plant_location'])
quality_prod['shift_encoded'] = le_shift.fit_transform(quality_prod['shift'])
quality_prod['product_encoded'] = le_product.fit_transform(quality_prod['product_sku'])

# Features
quality_features = ['plant_encoded', 'shift_encoded', 'product_encoded',
                   'raw_material_used_mt', 'processing_time_hours', 
                   'conversion_rate', 'moisture_level', 'oil_content']

X_quality = quality_prod[quality_features].fillna(quality_prod[quality_features].mean())
y_quality = quality_prod['brc_compliance_score']

# Split
X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(X_quality, y_quality, 
                                                              test_size=0.2, random_state=42)

# Train
rf_quality = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_quality.fit(X_train_q, y_train_q)

# Predict
y_pred_quality = rf_quality.predict(X_test_q)

# Evaluate
mae_quality = mean_absolute_error(y_test_q, y_pred_quality)
r2_quality = r2_score(y_test_q, y_pred_quality)

print(f"\n📊 BRC Score Prediction Performance:")
print(f"Mean Absolute Error: {mae_quality:.2f} points")
print(f"R² Score: {r2_quality:.3f}")

# Feature importance
quality_importance = pd.DataFrame({
    'feature': quality_features,
    'importance': rf_quality.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n📊 Top Quality Predictors:")
print(quality_importance)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Actual vs Predicted
axes[0].scatter(y_test_q, y_pred_quality, alpha=0.5)
axes[0].plot([y_test_q.min(), y_test_q.max()], [y_test_q.min(), y_test_q.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual BRC Score')
axes[0].set_ylabel('Predicted BRC Score')
axes[0].set_title('BRC Score Prediction', fontweight='bold')
axes[0].text(0.05, 0.95, f'R² = {r2_quality:.3f}\nMAE = {mae_quality:.2f}', 
             transform=axes[0].transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Feature importance
quality_importance.sort_values('importance').plot(kind='barh', x='feature', y='importance',
                                                   ax=axes[1], legend=False, color='green')
axes[1].set_xlabel('Importance')
axes[1].set_title('Quality Score Predictors', fontweight='bold')

plt.tight_layout()
plt.savefig('11_quality_prediction.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: 11_quality_prediction.png")
plt.close()



[6/8] Quality Score Prediction...

BRC COMPLIANCE PREDICTION

📊 BRC Score Prediction Performance:
Mean Absolute Error: 4.09 points
R² Score: -0.020

📊 Top Quality Predictors:
                 feature  importance
3   raw_material_used_mt    0.213458
5        conversion_rate    0.206699
6         moisture_level    0.152673
4  processing_time_hours    0.139195
7            oil_content    0.138407
2        product_encoded    0.085064
0          plant_encoded    0.034010
1          shift_encoded    0.030494

✓ Saved: 11_quality_prediction.png


In [11]:
# ============================================================================
# ANALYSIS 6: CITY EXPANSION PRIORITIZATION
# ============================================================================

print("\n[7/8] B2C City Expansion Analysis...")
print("\n" + "="*80)
print("CITY EXPANSION PRIORITIZATION")
print("="*80)

b2c_sales['sale_date'] = pd.to_datetime(b2c_sales['sale_date'])

# Calculate city metrics
city_metrics = b2c_sales.groupby('city').agg({
    'final_price': ['sum', 'mean', 'count'],
    'discount_percent': 'mean',
    'quantity_units': 'sum'
}).reset_index()

city_metrics.columns = ['city', 'total_revenue', 'avg_transaction', 'num_transactions',
                        'avg_discount', 'total_units']

# Calculate additional metrics
city_metrics['revenue_per_transaction'] = city_metrics['total_revenue'] / city_metrics['num_transactions']
city_metrics['growth_potential'] = (city_metrics['total_revenue'] / city_metrics['total_revenue'].max() * 100)

# Normalize for scoring
scaler = StandardScaler()
score_features = ['total_revenue', 'num_transactions', 'revenue_per_transaction']
city_metrics['expansion_score'] = scaler.fit_transform(city_metrics[score_features]).mean(axis=1)

city_metrics = city_metrics.sort_values('expansion_score', ascending=False)

print(f"\n📊 City Expansion Priority Ranking:")
for idx, row in city_metrics.iterrows():
    print(f"{row['city']}: Score {row['expansion_score']:.2f} | Revenue ₹{row['total_revenue']/100000:.2f}L | Transactions {row['num_transactions']:,}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Revenue by city
city_metrics.sort_values('total_revenue').plot(kind='barh', x='city', y='total_revenue',
                                                 ax=axes[0, 0], legend=False, color='teal')
axes[0, 0].set_xlabel('Total Revenue (₹)')
axes[0, 0].set_title('Revenue by City', fontweight='bold')

# Expansion score
city_metrics.sort_values('expansion_score').plot(kind='barh', x='city', y='expansion_score',
                                                   ax=axes[0, 1], legend=False, color='orange')
axes[0, 1].set_xlabel('Expansion Score')
axes[0, 1].set_title('City Expansion Priority', fontweight='bold')

# Revenue vs Transactions
axes[1, 0].scatter(city_metrics['num_transactions'], city_metrics['total_revenue']/100000, s=200, alpha=0.6)
for idx, row in city_metrics.iterrows():
    axes[1, 0].text(row['num_transactions'], row['total_revenue']/100000, row['city'], fontsize=9)
axes[1, 0].set_xlabel('Number of Transactions')
axes[1, 0].set_ylabel('Revenue (Lakhs)')
axes[1, 0].set_title('City Performance Matrix', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Avg discount by city
city_metrics.sort_values('avg_discount').plot(kind='barh', x='city', y='avg_discount',
                                                ax=axes[1, 1], legend=False, color='coral')
axes[1, 1].set_xlabel('Average Discount (%)')
axes[1, 1].set_title('Discount Strategy by City', fontweight='bold')

plt.tight_layout()
plt.savefig('12_city_expansion.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: 12_city_expansion.png")
plt.close()


[7/8] B2C City Expansion Analysis...

CITY EXPANSION PRIORITIZATION

📊 City Expansion Priority Ranking:
Gandhinagar: Score 1.24 | Revenue ₹48.55L | Transactions 18,496
Ahmedabad: Score 0.78 | Revenue ₹48.16L | Transactions 18,258
Anand: Score 0.00 | Revenue ₹47.98L | Transactions 18,255
Vadodara: Score -0.06 | Revenue ₹47.96L | Transactions 18,253
Surat: Score -0.28 | Revenue ₹48.01L | Transactions 18,337
Rajkot: Score -0.83 | Revenue ₹47.65L | Transactions 18,138
Bhavnagar: Score -0.86 | Revenue ₹47.73L | Transactions 18,206

✓ Saved: 12_city_expansion.png


In [12]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

# Final Summary
summary = f"""
1. CUSTOMER LIFETIME VALUE:
   - Model accuracy: {r2:.3f}
   - Average CLV: Rs. {customer_features['predicted_clv'].mean() / 100000:.2f} Lakhs
   - High-value customers: {(customer_features['clv_segment'] == 'Very High').sum()}

2. CHURN PREDICTION:
   - Current churn rate: {customer_features['churned'].mean() * 100:.2f}%
   - High-risk customers: {(customer_features['churn_risk'] == 'High').sum()}
   - Revenue at risk: Rs. {at_risk_valuable['total_revenue'].sum() / 10000000:.2f} Cr

3. DEMAND FORECASTING:
   - Forecast accuracy: {r2_demand:.3f}
   - Daily demand range: {y_demand.min():,.0f} - {y_demand.max():,.0f} kg
   - Prediction error: +/- {mae_demand:,.0f} kg

4. QUALITY PREDICTION:
   - Model accuracy: {r2_quality:.3f}
   - Prediction error: +/- {mae_quality:.2f} points
   - Key factor: {quality_importance.iloc[0]['feature']}

5. CITY EXPANSION:
   - Top city: {city_metrics.iloc[0]['city']}
   - Revenue potential: Rs. {city_metrics.iloc[0]['total_revenue']/100000:.2f} Lakhs
   - Current cities: {len(city_metrics)}
"""

print(summary)
print("\n" + "="*80)
print("ADVANCED ANALYTICS COMPLETE!")
print("="*80)


1. CUSTOMER LIFETIME VALUE:
   - Model accuracy: 0.980
   - Average CLV: Rs. 70.89 Lakhs
   - High-value customers: 50

2. CHURN PREDICTION:
   - Current churn rate: 48.00%
   - High-risk customers: 89
   - Revenue at risk: Rs. 15.30 Cr

3. DEMAND FORECASTING:
   - Forecast accuracy: -0.004
   - Daily demand range: 0 - 32,494 kg
   - Prediction error: +/- 4,590 kg

4. QUALITY PREDICTION:
   - Model accuracy: -0.020
   - Prediction error: +/- 4.09 points
   - Key factor: raw_material_used_mt

5. CITY EXPANSION:
   - Top city: Gandhinagar
   - Revenue potential: Rs. 48.55 Lakhs
   - Current cities: 7


ADVANCED ANALYTICS COMPLETE!


In [15]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to database
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'hyfun_analytics'
}

connection_string = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

# Check all tables
print("="*80)
print("DATA AVAILABILITY CHECK")
print("="*80)

tables = ['farmers_master', 'product_master', 'potato_procurement', 'production_batches',
          'quality_control', 'machine_downtime', 'wastage_tracking', 'b2b_customers',
          'b2b_orders', 'export_shipments', 'b2c_sales', 'revenue_summary']

for table in tables:
    count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table}", engine).iloc[0]['cnt']
    print(f"{table:<25} {count:>10,} records")
    
    # Check date ranges if applicable
    if 'date' in table or 'order' in table or 'sale' in table or 'procurement' in table or 'production' in table:
        date_col = None
        if table == 'b2b_orders':
            date_col = 'order_date'
        elif table == 'b2c_sales':
            date_col = 'sale_date'
        elif table == 'potato_procurement':
            date_col = 'procurement_date'
        elif table == 'production_batches':
            date_col = 'production_date'
        elif table == 'quality_control':
            date_col = 'inspection_date'
        elif table == 'revenue_summary':
            date_col = 'date'
        
        if date_col:
            dates = pd.read_sql(f"SELECT MIN({date_col}) as min_date, MAX({date_col}) as max_date FROM {table}", engine)
            print(f"  → Date range: {dates.iloc[0]['min_date']} to {dates.iloc[0]['max_date']}")

print("\n" + "="*80)

DATA AVAILABILITY CHECK
farmers_master                   500 records
product_master                    10 records
potato_procurement             3,262 records
  → Date range: 2022-01-01 to 2024-12-31
production_batches            12,528 records
  → Date range: 2022-01-01 to 2024-12-31
quality_control                3,727 records
machine_downtime               1,701 records
wastage_tracking               1,927 records
b2b_customers                    200 records
b2b_orders                     3,605 records
  → Date range: 2022-01-01 to 2024-12-31
export_shipments               3,338 records
b2c_sales                    127,943 records
  → Date range: 2022-01-01 to 2024-10-20
revenue_summary                5,463 records



In [16]:
# ============================================================================
# FIXED: DEMAND FORECASTING (Using ALL data, no recent filter)
# ============================================================================

print("\n[4/8] Building Demand Forecasting Model...")

# Load ALL orders
b2b_orders['order_date'] = pd.to_datetime(b2b_orders['order_date'])

# Aggregate to WEEKLY level (more stable than daily)
weekly_demand = b2b_orders.set_index('order_date').resample('W')['quantity_kg'].sum().reset_index()
weekly_demand.columns = ['date', 'quantity_kg']

# Only proceed if we have enough data
if len(weekly_demand) < 20:
    print(f"\n⚠️ Insufficient data for forecasting: Only {len(weekly_demand)} weeks available")
    print("Need at least 20 weeks. Skipping demand forecasting...")
else:
    # Create features
    weekly_demand['week_of_year'] = weekly_demand['date'].dt.isocalendar().week
    weekly_demand['month'] = weekly_demand['date'].dt.month
    weekly_demand['quarter'] = weekly_demand['date'].dt.quarter
    
    # Rolling statistics
    weekly_demand['rolling_mean_4'] = weekly_demand['quantity_kg'].rolling(window=4, min_periods=1).mean()
    weekly_demand['rolling_std_4'] = weekly_demand['quantity_kg'].rolling(window=4, min_periods=1).std()
    
    # Lag features
    weekly_demand['lag_1'] = weekly_demand['quantity_kg'].shift(1)
    weekly_demand['lag_4'] = weekly_demand['quantity_kg'].shift(4)
    
    # Drop NaN
    weekly_clean = weekly_demand.dropna()
    
    if len(weekly_clean) < 10:
        print(f"\n⚠️ After cleaning: Only {len(weekly_clean)} weeks available. Skipping...")
    else:
        # Features and target
        forecast_features = ['week_of_year', 'month', 'quarter', 'rolling_mean_4', 'rolling_std_4', 'lag_1', 'lag_4']
        X_demand = weekly_clean[forecast_features]
        y_demand = weekly_clean['quantity_kg']
        
        # Split (80-20)
        split_idx = int(len(X_demand) * 0.8)
        X_train_d = X_demand[:split_idx]
        X_test_d = X_demand[split_idx:]
        y_train_d = y_demand[:split_idx]
        y_test_d = y_demand[split_idx:]
        
        # Train
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.metrics import mean_absolute_error, r2_score
        
        rf_demand = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=8)
        rf_demand.fit(X_train_d, y_train_d)
        
        # Predict
        y_pred_demand = rf_demand.predict(X_test_d)
        
        # Evaluate
        mae_demand = mean_absolute_error(y_test_d, y_pred_demand)
        r2_demand = r2_score(y_test_d, y_pred_demand)
        
        print(f"\n📊 Weekly Demand Forecast Performance:")
        print(f"Mean Absolute Error: {mae_demand:,.0f} kg/week")
        print(f"R² Score: {r2_demand:.3f}")
        print(f"Average Weekly Demand: {y_demand.mean():,.0f} kg")
        
        # Visualization
        import matplotlib.pyplot as plt
        
        fig, ax = plt.subplots(1, 1, figsize=(14, 6))
        test_dates = weekly_clean['date'].iloc[split_idx:]
        ax.plot(test_dates.values, y_test_d.values, label='Actual', linewidth=2, marker='o')
        ax.plot(test_dates.values, y_pred_demand, label='Predicted', linewidth=2, marker='s')
        ax.set_title('Weekly Demand Forecast', fontsize=14, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel('Weekly Demand (kg)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('09_demand_forecasting.png', dpi=300, bbox_inches='tight')
        print("\n✓ Saved: 09_demand_forecasting.png")
        plt.close()


[4/8] Building Demand Forecasting Model...

📊 Weekly Demand Forecast Performance:
Mean Absolute Error: 10,571 kg/week
R² Score: -0.019
Average Weekly Demand: 61,569 kg

✓ Saved: 09_demand_forecasting.png


In [17]:
# ============================================================================
# ALTERNATIVE: QUALITY ANALYSIS (Descriptive, not predictive)
# ============================================================================

print("\n[6/8] Quality Control Analysis...")

# Merge quality with production
quality_prod = quality.merge(production[['batch_id', 'plant_location', 'product_sku']], on='batch_id')

# Analyze quality by plant
plant_quality = quality_prod.groupby('plant_location').agg({
    'brc_compliance_score': ['mean', 'std', 'min', 'max'],
    'defect_rate': 'mean',
    'qc_id': 'count'
}).round(2)

print("\n📊 Quality Performance by Plant:")
print(plant_quality)

# Analyze quality trends over time
quality['inspection_date'] = pd.to_datetime(quality['inspection_date'])
quality['month'] = quality['inspection_date'].dt.to_period('M')
monthly_quality = quality.groupby('month')['brc_compliance_score'].mean()

print("\n📊 Quality Trend (Last 12 months):")
print(monthly_quality.tail(12))

# Identify quality issues
poor_quality = quality[quality['brc_compliance_score'] < 85]
print(f"\n⚠️ Batches with poor quality (BRC < 85): {len(poor_quality)}")

# No ML model needed - just insights!


[6/8] Quality Control Analysis...

📊 Quality Performance by Plant:
                  brc_compliance_score                defect_rate qc_id
                                  mean   std min  max        mean count
plant_location                                                         
Ahmedabad Plant 1                92.77  4.68  85  100        2.77  1220
Ahmedabad Plant 2                92.46  4.69  85  100        2.74  1264
Rajkot Plant                     92.63  4.58  85  100        2.79  1243

📊 Quality Trend (Last 12 months):
month
2024-01    92.756303
2024-02    91.768421
2024-03    92.738739
2024-04    93.040000
2024-05    92.505051
2024-06    92.904762
2024-07    92.772727
2024-08    93.107843
2024-09    92.388889
2024-10    92.267241
2024-11    92.548387
2024-12    92.885965
Freq: M, Name: brc_compliance_score, dtype: float64

⚠️ Batches with poor quality (BRC < 85): 0
